# Triple Negative Breast Cancer

Triple-negative breast cancer (TNBC) accounts for about 10-15% of all breast cancers. These cancers tend to be more common in women younger than age 40, who are African-American.

Triple-negative breast cancer differs from other types of invasive breast cancer in that they grow and spread faster, have limited treatment options, and a worse prognosis (outcome). - American Cancer Society

Thus early stage cancer detection is required to provide proper treatment to the patient and reduce the risk of death due to cancer as detection of these cancer cells at later stages lead to more suffering and increases chances of death. Semantic segmentation of cancer cell images can be used to improvise the analysis and diagonsis of Breast Cancer! Below is such an attempt.

## U-Net

U-Net is a State of the Art CNN architecture for Bio-medical image segmentation. The architecture consists of a contracting path to capture context and a symmetric expanding path that enables precise localization. It's a Fully Convolutional Network(FCN) therefore it can work with arbitrary size images!

## Imports

In [ ]:
import os
import shutil
import random
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
print('Mounted successfully')

%matplotlib inline
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt, random, numpy as np, cv2
from PIL import Image
from tensorflow.keras import backend as K

from tensorflow.keras.preprocessing.image import ImageDataGenerator


Mounted at /content/drive
Mounted successfully


# Dataset Preparation


## Folder  Paths

In [ ]:
zip_file_path = '/content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation.zip'
extract_folder = '/content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation'

In [ ]:
!unzip -q "{zip_file_path}" -d "{extract_folder}"
os.listdir(extract_folder)

replace /content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/TNBC_NucleiSegmentation/GT_01/01_1.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

['TNBC_NucleiSegmentation', '__MACOSX', 'unet_weights.keras', 'train', 'test']

In [ ]:
base_path = "/content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/TNBC_NucleiSegmentation/"
destination_base_path = "/content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/"
train_path = os.path.join(destination_base_path, "train")
test_path = os.path.join(destination_base_path, "test")
train_ratio = 0.8

for folder in [train_path, test_path]:
    os.makedirs(os.path.join(folder, "images"), exist_ok=True)
    os.makedirs(os.path.join(folder, "label"), exist_ok=True)


## Extraction & Segregation

In [ ]:
for slide_folder in os.listdir(base_path):
    if slide_folder.lower().startswith("slide"):
        slide_path = os.path.join(base_path, slide_folder)
        gt_folder = "GT_" + slide_folder.split("_")[1]
        gt_path = os.path.join(base_path, gt_folder)

        if os.path.exists(gt_path):
            image_files = sorted([f for f in os.listdir(slide_path) if f.endswith(".png")])
            label_files = sorted([f for f in os.listdir(gt_path) if f.endswith(".png")])

            if len(image_files) == len(label_files):
                data_pairs = list(zip(image_files, label_files))
                random.shuffle(data_pairs)
                split_index = int(len(data_pairs) * train_ratio)
                train_data = data_pairs[:split_index]
                test_data = data_pairs[split_index:]

                slide_class = slide_folder.split("_")[1]
                class_train_path = os.path.join(train_path, f"images/{slide_class}")
                class_train_label_path = os.path.join(train_path, f"label/{slide_class}")
                class_test_path = os.path.join(test_path, f"images/{slide_class}")
                class_test_label_path = os.path.join(test_path, f"label/{slide_class}")

                os.makedirs(class_train_path, exist_ok=True)
                os.makedirs(class_train_label_path, exist_ok=True)
                os.makedirs(class_test_path, exist_ok=True)
                os.makedirs(class_test_label_path, exist_ok=True)

                for img_file, label_file in train_data:
                    img_path = os.path.join(slide_path, img_file)
                    label_path = os.path.join(gt_path, label_file)
                    shutil.copy(img_path, os.path.join(class_train_path, img_file))
                    shutil.copy(label_path, os.path.join(class_train_label_path, label_file))

                for img_file, label_file in test_data:
                    img_path = os.path.join(slide_path, img_file)
                    label_path = os.path.join(gt_path, label_file)
                    shutil.copy(img_path, os.path.join(class_test_path, img_file))
                    shutil.copy(label_path, os.path.join(class_test_label_path, label_file))
            else:
                print(f"Warning: Mismatch in number of images and labels in {slide_folder}")

print(f'Training images extracted in: {os.path.join(train_path, "images/")}')
print(f'Training labels extracted in: {os.path.join(train_path, "label/")}')
print(f'Testing images extracted in: {os.path.join(test_path, "images/")}')
print(f'Testing labels extracted in: {os.path.join(test_path, "label/")}')

Training images extracted in: /content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/train/images/
Training labels extracted in: /content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/train/label/
Testing images extracted in: /content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/test/images/
Testing labels extracted in: /content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/test/label/


# Helper Code

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: GPU device not found

### Model Code

In [ ]:
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, concatenate

def unet(pretrained_weights=None, input_size=(256, 256, 1)):
    """
    Initialises Keras Model instance. The following architecture is similar to the original U-Net
    architecture, except I've used "same" padding instead of "valid" which the authors have used.
    Using "same" padding throughout makes the output segmentation mask of the same (height, width)
    as that of the input.

    Args:
        pretrained_weights (.hdf5 file): Weights to pre-train our model
        input_size (tuple): Input shape of images to the model

    Returns:
        model (Model): Keras Model instance is the model we use
    """
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    # If pre-trained weights are provided, load them
    if pretrained_weights:
        model.load_weights(pretrained_weights)

    return model

### Augmentation Code


In [ ]:
def train_data_aug(canny=False):
    seed = 1

    image_datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.1,
    fill_mode='reflect',
    brightness_range=(0.8, 1.2),
    rescale=1./255)

    mask_datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    shear_range=0.1,
    fill_mode='reflect',
    brightness_range=(0.8, 1.2),
    rescale=1./255)

#    image_datagen = ImageDataGenerator(rotation_range=0.2, rescale=1./255, width_shift_range=0.05,
#                    height_shift_range=0.05, shear_range=0.05, zoom_range=0.05,
#                    horizontal_flip=True, fill_mode='nearest')
#    mask_datagen = ImageDataGenerator(rotation_range=0.2, rescale=1./255, width_shift_range=0.05,
#                    height_shift_range=0.05, shear_range=0.05, zoom_range=0.05,
#                    horizontal_flip=True, fill_mode='nearest')
    dir='/content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/train/'
    if canny:
        dir = 'train_canny/'
    image_generator = image_datagen.flow_from_directory(dir + 'images', class_mode=None, seed=seed,
                    color_mode="grayscale", target_size=(256, 256), batch_size=2)
    mask_generator = mask_datagen.flow_from_directory(dir + 'label', class_mode=None, seed=seed,
                    color_mode="grayscale", target_size=(256, 256), batch_size=2)

    for (img, mask) in zip(image_generator, mask_generator):
        yield (img, mask)



def test_data_aug(canny=False):
    seed = 1
    image_datagen1 = ImageDataGenerator(rescale=1./255)
    mask_datagen1 = ImageDataGenerator(rescale=1./255)
    dir = '/content/drive/My Drive/TNBC data/TNBC_NucleiSegmentation/test/'
    if canny:
        dir = 'test_canny/'
    image_generator1 = image_datagen1.flow_from_directory(dir + 'images', shuffle=False, class_mode=None,
                    seed=seed, color_mode="grayscale", target_size=(256, 256), batch_size=1)
    mask_generator1 = mask_datagen1.flow_from_directory(dir + 'label', shuffle=False, class_mode=None,
                    seed=seed, color_mode="grayscale", target_size=(256, 256), batch_size=1)

    # Yield pairs of (input, target)
    for (img, mask) in zip(image_generator1, mask_generator1):
        yield (img, mask)


### Metrics Code

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
	"""Computes Dice coefficient for y_true, y_pred

    Args:
        y_true (tensor): True data of shape (batch, 256, 256, 1)
        y_pred (tensor): Output/Prediction of our network of shape (batch, 256, 256, 1)
        smooth (float): To avoid division by 0
        See this stackoverflow discussion for explaination on y_true/y_pred: https://stackoverflow.com/a/46667294/11129457

    Returns:
        Computed Dice Coef (tensor): Dice coeffient computed on y_true and y_pred
	"""
	intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
	return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
	"""Computes Dice coefficient Loss which is used as the 'loss function' for training our network

    Args:
        y_true (tensor): True data of shape (batch, 256, 256, 1)
        y_pred (tensor): Output/Prediction of our network of shape (batch, 256, 256, 1)
        See this stackoverflow discussion for explaination on custom metrics: https://stackoverflow.com/a/45963039/11129457

    Returns:
        Computed Dice Coef Loss(tensor): Dice coeffient loss computed on y_true and y_pred
	"""
	return 1-dice_coef(y_true, y_pred)


def weighted_dice_loss(y_true, y_pred, beta=0.5, smooth=1e-6):
    intersection = K.sum(y_true * y_pred)
    union = K.sum(y_true) + K.sum(y_pred)
    dice = (2 * intersection + smooth) / (union + smooth)
    weight = beta if K.sum(y_true) < 0.5 else 1 - beta
    return weight * (1 - dice)

# Add Cross-Entropy for stability
combined_loss = lambda y_true, y_pred: weighted_dice_loss(y_true, y_pred) + tf.keras.losses.binary_crossentropy(y_true, y_pred)


def iou(y_true, y_pred, smooth=1):
	"""Computes Intersection-Over-Union which is used as a metric to judge our networks perfomance
		Check out this wonderful discussion on stats.stackexchange on F1 v/s iou: https://stats.stackexchange.com/a/276144

    Args:
        y_true (tensor): True data of shape (batch, 256, 256, 1)
        y_pred (tensor): Output/Prediction of our network of shape (batch, 256, 256, 1)
        smooth (float): To avoid division by 0

    Returns:
        Computed IOU(tensor): IOU computed on y_true and y_pred
	"""
	intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
	union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
	iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
	return iou

def F1(y_true, y_pred, smooth=1):
	"""Computes F1 Score which is used as a metric to judge our networks perfomance

    Args:
        y_true (tensor): True data of shape (batch, 256, 256, 1)
        y_pred (tensor): Output/Prediction of our network of shape (batch, 256, 256, 1)
        smooth (float): To avoid division by 0

    Returns:
        Computed F1(tensor): F1 computed on y_true and y_pred
	"""
	intersection = K.sum(y_true * y_pred, axis=[1,2,3])
	union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
	dice = K.mean((2. * intersection + smooth)/(union + smooth), axis=0)
	return dice

def recall(y_true, y_pred):
	"""Computes Recall which is used as a metric to judge our networks perfomance

    Args:
        y_true (tensor): True data of shape (batch, 256, 256, 1)
        y_pred (tensor): Output/Prediction of our network of shape (batch, 256, 256, 1)

    Returns:
        Computed Recall(tensor): Recall computed on y_true and y_pred
	"""
	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
	possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
	recall = true_positives / (possible_positives + K.epsilon())
	return recall

def precision(y_true, y_pred):
	"""Computes Precision which is used as a metric to judge our networks perfomance

    Args:
        y_true (tensor): True data of shape (batch, 256, 256, 1)
        y_pred (tensor): Output/Prediction of our network of shape (batch, 256, 256, 1)

    Returns:
        Computed Precision(tensor): Precision computed on y_true and y_pred
	"""
	true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
	predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
	precision = true_positives / (predicted_positives + K.epsilon())
	return precision


### Plots

In [ ]:
def training_history_plot(results):
	"""Plots "training curve" for the network/model for metrics listed below:
    		1. Dice loss
    		2. Pixel-wise accuracy
    		3. Intersection Over Union(IOU)
    		4. F1 score
    		5. Recall
    		6. Precision

    Args:
        results (History): Output of 'model.fit_generator()', 'History.history' attribute is a record of metrics
        					values as described above(from 1-6)

    Returns:
        None
	"""
	titles = ['Dice Loss','Accuracy','IOU','F1','Recall','Precision']
	metric = ['loss', 'accuracy', 'iou','f1','recall','precision'] # Metrics we're keeping track off

	# Define specification of our plot
	fig, axs = plt.subplots(3,2, figsize=(15, 15), facecolor='w', edgecolor='k')
	fig.subplots_adjust(hspace = 0.5, wspace=0.2)
	axs = axs.ravel()

	for i in range(6):
		axs[i].plot(results.history[metric[i]]) # Calls from 'History.history'- 'metric[i]', note 'results' is
		axs[i].set_title(titles[i])				# a 'History' object
		axs[i].set_xlabel('epoch')
		axs[i].set_ylabel(metric[i])
		axs[i].legend(['train'], loc='upper left')


def model_prediction_plot(results, all_results, t=0.2):
    """Displays:
        1. Original test image
        2. Network's predicted segmentation mask
        3. Binary mask obtained from 2
        4. Ground truth segmentation for the test image

    Args:
        results (numpy.array): Numpy array of predicted segmentation masks of shape (N, 256, 256, 1)
        all_results (list): List containing tuples of (class_name, image_name, predicted_mask)
        t (float)(Default=0.2): Threshold used to convert predicted mask to binary mask

    Returns:
        None
    """
    bin_result = (results >= t) * 1  # Convert predicted segmentation mask to binary mask on threshold `t`
    titles = ['Image', 'Predicted Mask', 'Binary Mask', 'Ground Truth']
    r = random.sample(range(len(results)), 4)  # Random sample for test images to display

    # Define specification of our plot
    fig, axs = plt.subplots(4, 4, figsize=(15, 15), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace=0.5, wspace=0.2)
    axs = axs.ravel()

    for i, idx in enumerate(r):  # Loop through random samples
        class_name, image_name = all_results[idx][:2]  # Get class and image name
        image_path = os.path.join(test_path, 'images', class_name, image_name)
        label_path = os.path.join(test_path, 'label', class_name, image_name)

        # Display test image
        axs[(i * 4) + 0].set_title(titles[0])
        image = Image.open(image_path).convert("L")
        axs[(i * 4) + 0].imshow(np.asarray(image) / 255, cmap='gray')

        # Display predicted segmentation mask
        axs[(i * 4) + 1].set_title(titles[1])
        pred_mask = np.squeeze(results[idx])
        axs[(i * 4) + 1].imshow(pred_mask, cmap="gray")

        # Display binary mask
        axs[(i * 4) + 2].set_title(titles[2])
        bin_mask = np.squeeze(bin_result[idx])
        axs[(i * 4) + 2].imshow(bin_mask, cmap="gray")

        # Display ground truth segmentation mask
        axs[(i * 4) + 3].set_title(titles[3])
        ground_truth = Image.open(label_path).convert("L")
        axs[(i * 4) + 3].imshow(np.asarray(ground_truth) / 255, cmap='gray')


def canny_compare_plot(results, results_canny):
	"""Compares model's performance on the "standard" dataset and dataset "overlayed" with "canny edges"
    	Displays:
    		1. Original test image
    		2. Predicted segmentation mask on "standard" dataset
    		3. Predicted segmentation mask on "overlayed" dataset
			4. Binary mask for "standard" dataset
			5. Binary mask for "overlayed" dataset
    		6. Ground truth segmentation for the test image

    Args:
        results (numpy.array): Numpy array of shape (17,255,255,1)- 17 predicted segmentation mask on "standard"
        						dataset, each of size (255,255,1)
        results_canny (numpy.array): Numpy array of shape (17,255,255,1)- 17 predicted segmentation mask on
        						"overlayed", each of size (255,255,1)

    Returns:
        None
	"""
	bin_result = (results >= 0.1) * 1 # Convert "standard" predicted segmentation mask to binary mask
	bin_result_canny = (results_canny >= 0.2) * 1 # Convert "overlayed" predicted segmentation mask to binary mask
	titles=['Image','Predicted Mask','Predicted Mask Canny','Binary Mask','Binary Mask Canny','Ground Truth']
	r=random.sample(range(17),4) # Random sample for test images to display

	# Define specification of our plot
	fig, axs = plt.subplots(4, 6, figsize=(15, 15), facecolor='w', edgecolor='k')
	fig.subplots_adjust(hspace = 0.5, wspace=0.2)
	axs = axs.ravel()

	for i in range(4): # 1 iteration for each selected test image
		# Displays test image
		axs[(i*6)+0].set_title(titles[0])
		fname = 'test/images/img/'+str(r[i])+'.png'
		image = Image.open(fname).convert("L")
		arr = np.asarray(image)
		axs[(i*6)+0].imshow(arr/255, cmap='gray')

		# Displays "standard" predicted segmentation mask
		axs[(i*6)+1].set_title(titles[1])
		I=np.squeeze(results[r[i],:,:,:])
		axs[(i*6)+1].imshow(I, cmap="gray")

		# Displays "overlayed" predicted segmentation mask
		axs[(i*6)+2].set_title(titles[3])
		I=np.squeeze(results_canny[r[i],:,:,:])
		axs[(i*6)+2].imshow(I, cmap="gray")

		# Displays "standard" binary mask
		axs[(i*6)+3].set_title(titles[2])
		I=np.squeeze(bin_result[r[i],:,:,:])
		axs[(i*6)+3].imshow(I, cmap="gray")

		# Displays "overlayed" binary mask
		axs[(i*6)+4].set_title(titles[4])
		I=np.squeeze(bin_result_canny[r[i],:,:,:])
		axs[(i*6)+4].imshow(I, cmap="gray")

		# Displays Ground truth segmentation mask
		axs[(i*6)+5].set_title(titles[5])
		fname = 'test/label/img/'+str(r[i])+'.png'
		image = Image.open(fname).convert("L")
		arr = np.asarray(image)
		axs[(i*6)+5].imshow(arr/255, cmap='gray')

def activation_map(image, layer, channel, m_c):
	"""Displays:
    		1. Original test image
    		2. Activation Map for provided layer and channel
    		3. Transparent overlay of Activation Map over test image

    Args:
        image (file name): Location of test image
        layer (int): Layer number, can be found from model summary
        channel (int): Channel number in the 'layer', number of channels in provided layer can be found from
        		model summary
        m_c (Model): Keras Model object used as network

    Returns:
        None
	"""

	fig, axs = plt.subplots(1, 3, figsize=(20, 20), facecolor='w', edgecolor='k')
	fig.subplots_adjust(wspace=0.2)
	axs = axs.ravel()

	ori=cv2.imread(image)
	axs[0].set_title('Original Image')
	axs[0].imshow(ori)


	img=cv2.imread(image, 0)
	x=cv2.resize(img, (256,256), interpolation = cv2.INTER_AREA)
	x=np.expand_dims(x, axis=2)
	x=np.expand_dims(x, axis=0)


	get_layer_output = K.function([m_c.layers[0].input, K.learning_phase()],
                                  [m_c.layers[layer].output])
	layer_output = get_layer_output([x, 0])[0]
	act=layer_output[0, :, :,channel]
	act=cv2.resize(act, (512,512))
	act=act/255

	axs[1].set_title('Activation Map')
	axs[1].imshow(act,cmap='jet')

	img=img.astype('float32')
	img=cv2.resize(img, (512,512))
	img=img/255
	dst=cv2.addWeighted(img,0.5,act,0.5,0)
	axs[2].set_title('Overlayed')
	axs[2].imshow(dst, cmap='jet')

### Utils Code

In [ ]:
def predict(class_name, image_name, m):
    """
    Predicts segmentation mask output for a single image.

    Args:
        class_name (str): Class name corresponding to the test directory.
        image_name (str): File name of the image to predict.
        m (Model): Model used to predict.

    Returns:
        result (numpy.array): Predicted segmentation mask for the image.
    """
    image_path = os.path.join(test_path, 'images', class_name, image_name)

    test_datagen = ImageDataGenerator(rescale=1./255)
    test_generator = test_datagen.flow_from_dataframe(
        dataframe=pd.DataFrame({"filename": [image_path]}),
        x_col="filename",
        y_col=None,
        class_mode=None,
        color_mode="grayscale",
        target_size=(256, 256),
        batch_size=1,
        shuffle=False
    )
    result = m.predict(test_generator, steps=1, verbose=1)
    return result


# Training

In [ ]:
m_c=unet()
m_c.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 1)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │            640 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 64)   │         36,928 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 128, 128, 128)  │        147,584 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        590,080 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 32, 32, 512)    │      2,359,808 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 32, 32, 512)    │              0 │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_3           │ (None, 16, 16, 512)    │              0 │ dropout[0][0]          │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 16, 16, 1024)   │      4,719,616 │ max_pooling2d_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_9 (Conv2D)         │ (None, 16, 16, 1024)   │      9,438,208 │ conv2d_8[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 16, 16, 1024)   │              0 │ conv2d_9[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d        

 Total params: 31,031,685 (118.38 MB)

 Trainable params: 31,031,685 (118.38 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.96
)
opt_c = Adam(lr_schedule, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
m_c.compile(loss=dice_coef_loss, optimizer=opt_c, metrics=['accuracy', iou, F1, recall, precision])



In [ ]:
checkpoint_c = ModelCheckpoint('unet_weights1.keras', monitor='loss',
                             verbose=1, save_best_only=True, mode='min') # Different checkpoint for storing "only" best wieghts during training
                                                                         # Weights will be saved in file named 'unet_canny_weights.hdf5'
train_generator_c=train_data_aug(False) # Peforms real-time Data Augmentation on the Canny Training dataset. See augmentation.py for more details
results_c = m_c.fit(train_generator_c, epochs=100, steps_per_epoch = 8, callbacks=[checkpoint_c])

NameError: name 'ModelCheckpoint' is not defined

In [ ]:
training_history_plot(results_c)

In [ ]:
titles = ['Dice Loss','Accuracy','IOU','F1','Recall','Precision']
test_generator_c=test_data_aug(False) # Peforms real-time Data Augmentation(here only re-scaling and converting to grayscale) on the Test/Validation Canny dataset. See augmentation.py for more details
performance_c=m_c.evaluate(test_generator_c, verbose=1,steps=17)

for i in range(6):
  print("%s = %f" %(titles[i], performance_c[i]))

In [ ]:
test_classes = sorted(os.listdir(os.path.join(test_path, "images")))

results = []

for class_name in test_classes:
    class_images_path = os.path.join(test_path, "images", class_name)
    class_images = sorted(os.listdir(class_images_path))

    for image_name in class_images:
        predicted_mask = predict(class_name, image_name, m_c)
        results.append((class_name, image_name, predicted_mask))

structured_results = np.array([r[2] for r in results])

In [ ]:
structured_results = np.squeeze(structured_results, axis=1)
print(structured_results.shape)
print(np.min(structured_results), np.max(structured_results))
binary_results = (structured_results > 0.3).astype(np.uint8)
print(binary_results.shape)

In [ ]:
model_prediction_plot(structured_results, results, t=0.5)